## Temp

In [4]:
# General imports here.
import pandas as pd
import numpy as np
import sklearn as sk
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
# Read the csv files and index appropriately.
ratings_small_df = pd.read_csv('./data/ratings_small.csv')
ratings_df = pd.read_csv('./data/ratings.csv')
links_small_df = pd.read_csv('./data/links_small.csv')
links_df = pd.read_csv('./data/links.csv')
keywords_df = pd.read_csv('./data/keywords.csv')
credits_df = pd.read_csv('./data/credits.csv')
metadata_df = pd.read_csv('./data/movies_metadata.csv')
# Reading the movies_metadata.csv file throws a DtypeWarning.
# TODO: Make sure that doesn't matter.

C:\Users\chris\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
dataframes = (ratings_small_df, ratings_df, links_small_df, links_df, keywords_df, credits_df, metadata_df)
# Pandas DataFrames don't have name attributes.
df_names = ("ratings_small", "ratings", "links_small", "links", "keywords", "credits", "movies_metadata")

In [33]:
for df, name in zip(dataframes, df_names):
    print("\n" + "Dataframe: " + name + "\n")
    print(df.describe())


Dataframe: ratings_small

              userId        movieId         rating     timestamp
count  100004.000000  100004.000000  100004.000000  1.000040e+05
mean      347.011310   12548.664363       3.543608  1.129639e+09
std       195.163838   26369.198969       1.058064  1.916858e+08
min         1.000000       1.000000       0.500000  7.896520e+08
25%       182.000000    1028.000000       3.000000  9.658478e+08
50%       367.000000    2406.500000       4.000000  1.110422e+09
75%       520.000000    5418.000000       4.000000  1.296192e+09
max       671.000000  163949.000000       5.000000  1.476641e+09

Dataframe: ratings

             userId       movieId        rating     timestamp
count  2.602429e+07  2.602429e+07  2.602429e+07  2.602429e+07
mean   1.350371e+05  1.584911e+04  3.528090e+00  1.171258e+09
std    7.817620e+04  3.108526e+04  1.065443e+00  2.052889e+08
min    1.000000e+00  1.000000e+00  5.000000e-01  7.896520e+08
25%    6.716400e+04  1.073000e+03  3.000000e+00  9.907545